This project tries to classify the data from EEG signals as high Valance or low Valance.

In [84]:
import pandas as pd
import numpy as np
import pickle
import math
import statistics as stat
import scipy.stats as scStat

### The DEAP Dataset

The dataset contains 40 experiments for each of the 32 participants. The labels array contain the valence, arousal, dominance and liking ratings for each participant for each of the 40 experiments. The data array contains 8064 physiological/EEG signal data from 40 different channels for each of the 40 experiments for each of the 32 participants.

### Feature Extraction

We divide the 8064 readings per channel, into 10 batches of approximately 807 readings each. For each batch we extract the mean, median, maximum, minimum, standard deviation, variance, range, skewness and kurtosis values for the 807 readings. Hence for each of the 10 batches of a single channel we extract 9 values mentioned above, we get 90 values as our processed dataset. We further add the net mean, median, maximum, minimum, standard deviation, variance, range, skewness and kurtosis values for the entire 8064 readings along with the experiment and participant number to our dataset, bringing it up to 101 values per channel.

In [124]:
def extract_features(data, trial, participantNumber):
    extData = []
    for x in np.array_split(data, 10):
        extData.extend(calc_features(x))
    extData.extend(calc_features(data))
    extData.append(participantNumber)
    extData.append(participantNumber)
    return extData

In [94]:
def calc_features(array):
    return [stat.mean(array),
                stat.median(array),
                stat.variance(array),
                stat.stdev(array),
                max(array),
                min(array),
                scStat.mode(array)[0][0],
                scStat.kurtosis(array),
                scStat.skew(array, axis=0, bias=True)]

Features from each channel are extracted and appended to a df so that it can be stored into a csv file and accessed later.

In [258]:
def process_data_file(fileName, participantNumber):
    with open(fileName, 'rb') as f: content = pickle.load(f, encoding='latin1')
    data = content['data']
    labels = content['labels']
    extracted_features = []
    for index, trialData in enumerate(data):
        for i, channelData in enumerate(trialData):
            extracted_features.append(extract_features(channelData, index, participantNumber))
    df = pd.DataFrame(extracted_features)
    df['Valance Label'] = list(labels[:,0])*int(len(df)/len(labels))
    df['Arousal Label'] = list(labels[:,1])*int(len(df)/len(labels))
    return df

### Reading Data from DEAP Dataset

The data from DEAP Dataset .dat files are read one by one and the extracted features are appended into a csv file.

In [261]:
files = ['s01.dat', 's02.dat', 's03.dat']
participants = [1, 2, 3]
for f, participantNumber in zip(files, participants):
    process_data_file(f, participantNumber).to_csv('ExtractedFeatures.csv', mode='a', index=False, header=False)

### Learning from extracted features

If you're reading from the provided csv file, learning starts here

In [326]:
from sklearn.preprocessing import OneHotEncoder

In [329]:
columns = [str(i) for i in range(0,99)]
columns.extend(['experiment No', 'participant No', 'Valance Label', 'Arousal Label'])
df = pd.read_csv('ExtractedFeatures.csv', header=None, names=columns)

In [328]:
df['Valance Label'] = df['Valance Label'].apply(lambda x: 1 if x>5 else 0)
df['Arousal Label'] = df['Arousal Label'].apply(lambda x: 1 if x>5 else 0)

In [316]:
X = df[df.columns.difference(['Valance Label', 'Arousal Label'])].values

y_valance = df['Valance Label'].values
y_arousal = df['Arousal Label'].values

In [319]:
X = X.reshape(120,40,101)

In [330]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [325]:
model = Sequential()
model.add(Conv2D(100, (3,3), padding="valid", activation='tanh', input_shape = (40, 101, 1)))
model.add(Conv2D(100, (3, 3), activation='tanh' ))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.50))
model.add(Dense(2, activation='softplus'))

sgd = SGD(lr = 0.00001, momentum = 0.9, nesterov = True)
model.compile(loss ='categorical_crossentropy', optimizer = sgd,metrics=['accuracy'])

In [313]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 38, 99, 100)       1000      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 36, 97, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 48, 100)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 48, 100)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 86400)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               11059328  
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [336]:
y_valance = to_categorical(y_valance, num_classes=2)
y_arousal = to_categorical(y_arousal, num_classes=2)

In [337]:
kf = KFold(n_splits=3, shuffle=False)
losses = []
accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_valance[train_index], y_valance[test_index]
    
    #print('train: ', X_train.shape, y_train.shape)
    X_train = X_train.reshape(X_train.shape[0], 40, 101, 1)
    X_test = X_test.reshape(X_test.shape[0], 40, 101, 1)
    
    history = model.fit(X_train, y_train, batch_size = 50, 
                        epochs = 250, validation_data = (X_test, y_test))
    score = model.evaluate(X_test, y_test, batch_size = 1)
    losses.append(score[0])
    accuracies.append(score[1])

Train on 80 samples, validate on 40 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/250
80/80 [==============================] - 8s 102ms/sample - loss: 0.8504 - acc: 0.5125 - val_loss: 0.6891 - val_acc: 0.5250
Epoch 2/250
80/80 [==============================] - 7s 81ms/sample - loss: 0.8051 - acc: 0.5125 - val_loss: 0.6821 - val_acc: 0.5000
Epoch 3/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.8713 - acc: 0.4375 - val_loss: 0.6812 - val_acc: 0.5750
Epoch 4/250
80/80 [==============================] - 6s 71ms/sample - loss: 0.8445 - acc: 0.4625 - val_loss: 0.6878 - val_acc: 0.5000
Epoch 5/250
80/80 [==============================] - 7s 83ms/sample - loss: 0.7524 - acc: 0.5500 - val_loss: 0.7001 - val_acc: 0.5250
Epoch 6/250
80/80 [==============================] - 6s 77ms/sample - loss: 0.7819 - acc: 0.5500 - val_loss: 0.7115 - val_acc: 0.5000
Epoch 7/250
80/80 [==============================] - 6s 7

Epoch 59/250
80/80 [==============================] - 6s 73ms/sample - loss: 0.7352 - acc: 0.4625 - val_loss: 0.6862 - val_acc: 0.5750
Epoch 60/250
80/80 [==============================] - 6s 77ms/sample - loss: 0.7159 - acc: 0.4750 - val_loss: 0.6863 - val_acc: 0.6250
Epoch 61/250
80/80 [==============================] - 6s 69ms/sample - loss: 0.7084 - acc: 0.5125 - val_loss: 0.6863 - val_acc: 0.6250
Epoch 62/250
80/80 [==============================] - 6s 73ms/sample - loss: 0.7247 - acc: 0.5250 - val_loss: 0.6862 - val_acc: 0.6250
Epoch 63/250
80/80 [==============================] - 5s 68ms/sample - loss: 0.7026 - acc: 0.5500 - val_loss: 0.6864 - val_acc: 0.5750
Epoch 64/250
80/80 [==============================] - 6s 73ms/sample - loss: 0.7137 - acc: 0.5625 - val_loss: 0.6865 - val_acc: 0.5500
Epoch 65/250
80/80 [==============================] - 6s 75ms/sample - loss: 0.7140 - acc: 0.4625 - val_loss: 0.6862 - val_acc: 0.5500
Epoch 66/250
80/80 [==============================] - 6

80/80 [==============================] - 5s 68ms/sample - loss: 0.6524 - acc: 0.6125 - val_loss: 0.6863 - val_acc: 0.4750
Epoch 120/250
80/80 [==============================] - 6s 73ms/sample - loss: 0.6349 - acc: 0.6125 - val_loss: 0.6863 - val_acc: 0.4750
Epoch 121/250
80/80 [==============================] - 6s 77ms/sample - loss: 0.6743 - acc: 0.6125 - val_loss: 0.6862 - val_acc: 0.4750
Epoch 122/250
80/80 [==============================] - 6s 72ms/sample - loss: 0.6795 - acc: 0.6125 - val_loss: 0.6862 - val_acc: 0.4750
Epoch 123/250
80/80 [==============================] - 6s 81ms/sample - loss: 0.6555 - acc: 0.6250 - val_loss: 0.6862 - val_acc: 0.4750
Epoch 124/250
80/80 [==============================] - 6s 77ms/sample - loss: 0.6897 - acc: 0.5000 - val_loss: 0.6858 - val_acc: 0.4750
Epoch 125/250
80/80 [==============================] - 5s 68ms/sample - loss: 0.6524 - acc: 0.5625 - val_loss: 0.6860 - val_acc: 0.4750
Epoch 126/250
80/80 [==============================] - 6s 73ms

80/80 [==============================] - 5s 67ms/sample - loss: 0.5645 - acc: 0.7625 - val_loss: 0.6858 - val_acc: 0.5000
Epoch 180/250
80/80 [==============================] - 6s 81ms/sample - loss: 0.6173 - acc: 0.6750 - val_loss: 0.6867 - val_acc: 0.5000
Epoch 181/250
80/80 [==============================] - 6s 81ms/sample - loss: 0.6388 - acc: 0.6125 - val_loss: 0.6875 - val_acc: 0.5000
Epoch 182/250
80/80 [==============================] - 6s 80ms/sample - loss: 0.6838 - acc: 0.5500 - val_loss: 0.6882 - val_acc: 0.5000
Epoch 183/250
80/80 [==============================] - 7s 92ms/sample - loss: 0.6488 - acc: 0.6125 - val_loss: 0.6890 - val_acc: 0.5000
Epoch 184/250
80/80 [==============================] - 7s 83ms/sample - loss: 0.6703 - acc: 0.5875 - val_loss: 0.6896 - val_acc: 0.5000
Epoch 185/250
80/80 [==============================] - 6s 73ms/sample - loss: 0.6320 - acc: 0.6625 - val_loss: 0.6903 - val_acc: 0.5000
Epoch 186/250
80/80 [==============================] - 6s 75ms

80/80 [==============================] - 6s 81ms/sample - loss: 0.6000 - acc: 0.6250 - val_loss: 0.7093 - val_acc: 0.5000
Epoch 240/250
80/80 [==============================] - 8s 98ms/sample - loss: 0.6201 - acc: 0.6750 - val_loss: 0.7111 - val_acc: 0.5000
Epoch 241/250
80/80 [==============================] - 8s 95ms/sample - loss: 0.6048 - acc: 0.6875 - val_loss: 0.7134 - val_acc: 0.5000
Epoch 242/250
80/80 [==============================] - 8s 98ms/sample - loss: 0.5641 - acc: 0.7125 - val_loss: 0.7155 - val_acc: 0.5000
Epoch 243/250
80/80 [==============================] - 7s 91ms/sample - loss: 0.6221 - acc: 0.6500 - val_loss: 0.7162 - val_acc: 0.5250
Epoch 244/250
80/80 [==============================] - 8s 100ms/sample - loss: 0.5883 - acc: 0.6625 - val_loss: 0.7174 - val_acc: 0.5250
Epoch 245/250
80/80 [==============================] - 8s 98ms/sample - loss: 0.6296 - acc: 0.5875 - val_loss: 0.7166 - val_acc: 0.5250
Epoch 246/250
80/80 [==============================] - 7s 93m

...

80/80 [==============================] - 6s 80ms/sample - loss: 0.3819 - acc: 0.8125 - val_loss: 0.8589 - val_acc: 0.4750
Epoch 230/250
80/80 [==============================] - 8s 95ms/sample - loss: 0.4227 - acc: 0.8250 - val_loss: 0.9343 - val_acc: 0.4750
Epoch 231/250
80/80 [==============================] - 6s 80ms/sample - loss: 0.4276 - acc: 0.7875 - val_loss: 0.9507 - val_acc: 0.4750
Epoch 232/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.4282 - acc: 0.8375 - val_loss: 0.9631 - val_acc: 0.4750
Epoch 233/250
80/80 [==============================] - 8s 97ms/sample - loss: 0.4376 - acc: 0.8125 - val_loss: 0.9405 - val_acc: 0.4750
Epoch 234/250
80/80 [==============================] - 7s 90ms/sample - loss: 0.3671 - acc: 0.8375 - val_loss: 0.9057 - val_acc: 0.4750
Epoch 235/250
80/80 [==============================] - 6s 80ms/sample - loss: 0.4691 - acc: 0.7875 - val_loss: 0.8429 - val_acc: 0.4750
Epoch 236/250
80/80 [==============================] - 6s 81ms

Epoch 39/250
80/80 [==============================] - 8s 96ms/sample - loss: 0.5444 - acc: 0.7500 - val_loss: 0.2954 - val_acc: 0.9250
Epoch 40/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.4598 - acc: 0.7500 - val_loss: 0.3092 - val_acc: 0.8750
Epoch 41/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.4401 - acc: 0.8000 - val_loss: 0.3124 - val_acc: 0.8500
Epoch 42/250
80/80 [==============================] - 6s 81ms/sample - loss: 0.5066 - acc: 0.7875 - val_loss: 0.3050 - val_acc: 0.9000
Epoch 43/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.4255 - acc: 0.7625 - val_loss: 0.2948 - val_acc: 0.9250
Epoch 44/250
80/80 [==============================] - 7s 87ms/sample - loss: 0.4737 - acc: 0.8250 - val_loss: 0.2938 - val_acc: 0.9250
Epoch 45/250
80/80 [==============================] - 7s 88ms/sample - loss: 0.4636 - acc: 0.8125 - val_loss: 0.2842 - val_acc: 0.9250
Epoch 46/250
80/80 [==============================] - 7

80/80 [==============================] - 8s 99ms/sample - loss: 0.3645 - acc: 0.8625 - val_loss: 0.2933 - val_acc: 0.9250
Epoch 100/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.4520 - acc: 0.7750 - val_loss: 0.3029 - val_acc: 0.9250
Epoch 101/250
80/80 [==============================] - 8s 96ms/sample - loss: 0.4154 - acc: 0.8125 - val_loss: 0.3290 - val_acc: 0.8750
Epoch 102/250
80/80 [==============================] - 7s 83ms/sample - loss: 0.4062 - acc: 0.8125 - val_loss: 0.3438 - val_acc: 0.8750
Epoch 103/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.4123 - acc: 0.8375 - val_loss: 0.3442 - val_acc: 0.8750
Epoch 104/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.4081 - acc: 0.8125 - val_loss: 0.3467 - val_acc: 0.8750
Epoch 105/250
80/80 [==============================] - 7s 92ms/sample - loss: 0.3615 - acc: 0.8875 - val_loss: 0.3474 - val_acc: 0.8750
Epoch 106/250
80/80 [==============================] - 7s 87ms

80/80 [==============================] - 7s 86ms/sample - loss: 0.3716 - acc: 0.8375 - val_loss: 0.4349 - val_acc: 0.7250
Epoch 160/250
80/80 [==============================] - 8s 96ms/sample - loss: 0.3662 - acc: 0.8375 - val_loss: 0.3990 - val_acc: 0.7750
Epoch 161/250
80/80 [==============================] - 7s 90ms/sample - loss: 0.4397 - acc: 0.8000 - val_loss: 0.3729 - val_acc: 0.8250
Epoch 162/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.3491 - acc: 0.8875 - val_loss: 0.3570 - val_acc: 0.8500
Epoch 163/250
80/80 [==============================] - 7s 90ms/sample - loss: 0.3878 - acc: 0.8375 - val_loss: 0.3469 - val_acc: 0.8750
Epoch 164/250
80/80 [==============================] - 7s 86ms/sample - loss: 0.3964 - acc: 0.8500 - val_loss: 0.3632 - val_acc: 0.8500
Epoch 165/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.3341 - acc: 0.8875 - val_loss: 0.3609 - val_acc: 0.8500
Epoch 166/250
80/80 [==============================] - 8s 100m

80/80 [==============================] - 7s 87ms/sample - loss: 0.3160 - acc: 0.8875 - val_loss: 0.4706 - val_acc: 0.7250
Epoch 220/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.2689 - acc: 0.9125 - val_loss: 0.4641 - val_acc: 0.7250
Epoch 221/250
80/80 [==============================] - 7s 91ms/sample - loss: 0.3759 - acc: 0.8250 - val_loss: 0.4527 - val_acc: 0.7500
Epoch 222/250
80/80 [==============================] - 7s 90ms/sample - loss: 0.2995 - acc: 0.8875 - val_loss: 0.4387 - val_acc: 0.7500
Epoch 223/250
80/80 [==============================] - 7s 87ms/sample - loss: 0.3045 - acc: 0.9125 - val_loss: 0.4161 - val_acc: 0.7750
Epoch 224/250
80/80 [==============================] - 8s 104ms/sample - loss: 0.3470 - acc: 0.8500 - val_loss: 0.4069 - val_acc: 0.7750
Epoch 225/250
80/80 [==============================] - 7s 89ms/sample - loss: 0.2803 - acc: 0.9000 - val_loss: 0.3942 - val_acc: 0.8250
Epoch 226/250
80/80 [==============================] - 8s 95m

In [345]:
np.array(losses).mean()

0.6936688152762751